# A failed attempt at reproducing Fig. 1 of [Sergei Nayakshin 2005](https://doi.org/10.1111/j.1365-2966.2005.08913.x)!


In [1]:
import time
import torch # so the codes below can run on GPU.
import numpy 
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

%matplotlib inline

# Equation 3

In [86]:
def cos_lambda(phi, phi_one, beta = 3.14/4.):
    
    output = numpy.cos(beta) * numpy.sin(phi) * numpy.sin(phi_one) + numpy.cos(phi) * numpy.cos(phi_one)
    
    return output
    

# Equation 6

In [82]:
def delta(r_one = 2., r = 2.):
    numerator   = 2. * r_one * r
    denominator = r_one ** 2 + r ** 2
    ratio       = numerator/denominator
    return ratio

# Equation 5

In [98]:
# https://math.stackexchange.com/questions/2121611/simpson-rule-for-double-integral

def integral_delta_beta(inner_step_size, outer_step_size, a_start, b_end, c_start, d_end, beta = 3.14/4.):
    
    j         = c_start
    current_c = c_start
    current_d = c_start + outer_step_size
    
    i         = a_start
    current_a = a_start
    current_b = a_start + inner_step_size
    
    coeff     = (1/36) * inner_step_size * outer_step_size
    
    incremental_sum = []
    while j < d_end:
        i         = a_start
        current_a = a_start
        current_b = a_start + outer_step_size
        while i < b_end:
            term_one   = 16 * (numpy.sin((current_a + current_b)/2) * numpy.sin((current_c + current_d)/2)/ \
                                 (1 - delta() * cos_lambda((current_a + current_b)/2, \
                                                           ((current_c + current_d)/2)))**(3/2))
            
            term_two   = 4  * ((numpy.sin((current_a + current_b)/2) * numpy.sin(current_d))/ \
                                 (1 - delta() * cos_lambda((current_a + current_b)/2, current_d))**(3/2))
            
            term_three = 4  * ((numpy.sin((current_a + current_b)/2) * numpy.sin(current_c))/ \
                                 (1 - delta() * cos_lambda((current_a + current_b)/2, current_c))**(3/2))
            
            term_four  = 4  * ((numpy.sin(current_b) * numpy.sin((current_c + current_d)/2))/ \
                                 (1 - delta() * cos_lambda(current_b, (current_c + current_d)/2))**(3/2))
            
            term_five  =      ((numpy.sin(current_b) * numpy.sin(current_d))/ \
                                 (1 - delta() * cos_lambda(current_b, current_d))**(3/2))
            
            term_six   =      ((numpy.sin(current_b) * numpy.sin(current_c))/ \
                                 (1 - delta() * cos_lambda(current_b, current_c))**(3/2))
            
            term_seven = 4  * ((numpy.sin(current_a) * numpy.sin((current_c + current_d)/2))/ \
                                 (1 - delta() * cos_lambda(current_a, (current_c + current_d)/2))**(3/2))
            
            term_eight =      ((numpy.sin(current_a) * numpy.sin(current_d))/ \
                                 (1 - delta() * cos_lambda(current_a, current_d))**(3/2))
            
            term_nine  =      ((numpy.sin(current_a) * numpy.sin(current_c))/ \
                                 (1 - delta() * cos_lambda(current_a, current_c))**(3/2))
            
            # Adding all the terms
            rectangle_result = coeff * (term_one + term_two + term_three + term_four + term_five + term_six + \
                                        term_seven + term_eight + term_nine)
            # Updateing the begining and end of the inner interval for the next iteration
            current_a = current_b
            current_b = current_b + inner_step_size
        
            # Appending results to the list
            incremental_sum.append(rectangle_result)
            
            # Updating i
            i += inner_step_size

        # Updateing the begining and end of the inner interval for the next iteration
        current_c = current_d
        current_d = current_d + outer_step_size
        
        # Updating j
        j += outer_step_size

    increment_array    = numpy.array(incremental_sum)
    integral_output    = (numpy.sin(beta)/(4 * numpy.pi**2)) * increment_array.sum()
    
    return integral_output

# Equation 4

In [108]:
def torque_21(delta_beta, m_one = 1000, r_one = 2, r=2, m = 0.001, g_const = (6.6743)**(-11)):
    
    numerator   = g_const * m_one * m * r_one * r
    denominator = (r_one ** 2 + r ** 2)**(3/2)
    output      = (numerator/denominator) * delta_beta
    
    return output
    

Below we calculate $\Omega_k$ (total_angular_momentum) to be used in the denominator of eq. 7.

In [119]:
# https://math.stackexchange.com/questions/900936/angular-momentum-of-an-accretion-disk
# https://www.ita.uni-heidelberg.de/~mattia/teaching/accretion_disks.pdf

def total_angular_momentum(g_const = (6.6743)**(-11), m = 0.001, r = 2):
    
    output = numpy.sqrt(((g_const * m)/r**3))
    
    return output
    

# Equation 7

In [113]:
def omega_p(torque_21_value, total_ang_momentum = total_angular_momentum(), m = 0.001, r = 2, beta = 3.14/4.):
    
    numerator   = torque_21_value
    denominator = m * total_ang_momentum * (r**2) * numpy.sin(beta)
    
    output      = numerator/denominator
    
    return output
    

# Equation 13

In [114]:
def gamma_angle(omega_p, time):
    
    output = 3.14/2 + omega_p * time
    
    return output
    

# Equation 17, 18, and 19

In [115]:
def x_y_z_prime(gamma, phi, r = 2, beta = 3.14/4.):
    
    x_prime = (numpy.cos(beta)*numpy.cos(gamma)*numpy.sin(phi) + numpy.sin(gamma)*numpy.cos(phi)) * r
    
    y_prime = (-numpy.cos(beta)*numpy.cos(gamma)*numpy.cos(phi) + numpy.sin(phi) * \
                ((numpy.cos(beta)**2) * numpy.sin(gamma) + numpy.sin(beta)**2)) * r
    
    z_prime = (-numpy.sin(beta) * (numpy.cos(gamma)*numpy.cos(phi) + (1 - numpy.sin(gamma)) * numpy.cos(beta) * \
                                  numpy.sin(phi)))* r
    
    return x_prime, y_prime, z_prime
    

In [117]:
start = time.time()
inner_start = 0.0001
inner_end   = 6.28

outer_start = 0.0001
outer_end   = 6.28

inner_step = (inner_end - inner_start)/500
outer_step = (outer_end - outer_start)/500

test_run_torque = integral_delta_beta(inner_step, outer_step, inner_start, inner_end, outer_start, outer_end)
test_run_torque

end = time.time()
print(end - start)

0.842639501611494

The problem is that equations 17, 18, and 19 depend on equations 14, 15, and 16 for which I do not have the angles $\alpha$, $\beta$ and $\gamma$! My knowledge of classical mechanics is rusty! I need to get back to it later!!  